In [2]:
import os

import time
import mujoco
import mujoco_viewer
import numpy as np

base_dir = "/Users/nikita/Documents/Учеба/ИМРС/simrobs_group_2025/SRS/practice_3/submissions"
xml_path = os.path.join(base_dir, "335259_MikhinNikita_Task3", "optimus.xml")

print("XML path:", xml_path)

model = mujoco.MjModel.from_xml_path(xml_path)
data = mujoco.MjData(model)

print("nq =", model.nq, " nv =", model.nv)

data.qpos[:] = 0.0

if model.nq >= 3:
    data.qpos[0] = np.deg2rad(-35.0)
    data.qpos[1] = np.deg2rad(25.0)
    data.qpos[2] = np.deg2rad(-10.0)
elif model.nq >= 2:
    data.qpos[0] = np.deg2rad(-35.0)
    data.qpos[1] = np.deg2rad(25.0)

mujoco.mj_forward(model, data)

SIMEND = 10.0
TIMESTEP = 0.001
STEP_NUM = int(SIMEND / TIMESTEP)
times = np.linspace(0, SIMEND, STEP_NUM)

try:
    knee_site_id = mujoco.mj_name2id(
        model, mujoco.mjtObj.mjOBJ_SITE, "sB"
    )
    if knee_site_id < 0:
        raise ValueError("site 'sB' not found")
except Exception as e:
    print("WARNING:", e)
    knee_site_id = None

knee_x = []
knee_z = []

viewer = mujoco_viewer.MujocoViewer(
    model,
    data,
    title="Optimus knee mechanism",
    width=1280,
    height=720,
)

RUN_SIM = True

for i in range(STEP_NUM):
    if not viewer.is_alive:
        break

    if RUN_SIM:
        time.sleep(0.01)
        mujoco.mj_step(model, data)

        if knee_site_id is not None:
            pos_knee = data.site_xpos[knee_site_id]
            knee_x.append(pos_knee[0])
            knee_z.append(pos_knee[2])

    viewer.render()

viewer.close()

mid = STEP_NUM // 2


def model_fn():
    return mujoco.MjModel.from_xml_path(xml_path)


def data_fn(m: mujoco.MjModel) -> mujoco.MjData:
    d = mujoco.MjData(m)
    d.qpos[:] = 0.0
    if m.nq >= 3:
        d.qpos[0] = np.deg2rad(-35.0)
        d.qpos[1] = np.deg2rad(25.0)
        d.qpos[2] = np.deg2rad(-10.0)
    elif m.nq >= 2:
        d.qpos[0] = np.deg2rad(-35.0)
        d.qpos[1] = np.deg2rad(25.0)
    mujoco.mj_forward(m, d)
    return d


def viewer_fn(m: mujoco.MjModel,
              d: mujoco.MjData,
              sim_time: float = 5.0,
              dt: float = 0.001,
              run_sim: bool = True,
              title: str = "Optimus - viewer_fn"):
    steps = int(sim_time / dt)
    v = mujoco_viewer.MujocoViewer(m, d, title=title)
    for _ in range(steps):
        if not v.is_alive:
            break
        if run_sim:
            mujoco.mj_step(m, d)
        v.render()
    v.close()

XML path: /Users/nikita/Documents/Учеба/ИМРС/simrobs_group_2025/SRS/practice_3/submissions/335259_MikhinNikita_Task3/optimus.xml
nq = 2  nv = 2
